In [1]:
import os
import sqlite3
from flask import Flask, render_template, send_from_directory

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'pdfs'
app.config['DATABASE'] = 'pdfs.db'

# Define the database schema and create a connection
def create_db():
    conn = sqlite3.connect(app.config['DATABASE'])
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS pdf_files
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  filename TEXT NOT NULL,
                  path TEXT NOT NULL)''')
    conn.commit()
    conn.close()

# Add PDF files to the database
def add_pdf_files_to_db():
    conn = sqlite3.connect(app.config['DATABASE'])
    c = conn.cursor()
    for filename in os.listdir(app.config['UPLOAD_FOLDER']):
        if filename.endswith('.pdf'):
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            secure_filename = filename.replace(' ', '_')
            c.execute("INSERT INTO pdf_files (filename, path) VALUES (?, ?)",
                      (secure_filename, filepath))
    conn.commit()
    conn.close()

# Retrieve PDF files from the database
def get_pdf_files_from_db():
    conn = sqlite3.connect(app.config['DATABASE'])
    c = conn.cursor()
    c.execute("SELECT filename, path FROM pdf_files")
    pdf_files = c.fetchall()
    conn.close()
    return pdf_files

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/pdfs')
def pdfs():
    pdf_files = get_pdf_files_from_db()
    return render_template('pdfs.html', pdf_files=pdf_files)

@app.route('/pdfs/<filename>')
def serve_pdf(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

if __name__ == '__main__':
    create_db()
    add_pdf_files_to_db()
    app.run()


OperationalError: table pdf_files has no column named path

In [1]:
import os
from flask import Flask, render_template, send_from_directory, url_for, g
from werkzeug.utils import secure_filename
import sqlite3

app = Flask(__name__)

# Define the directory where PDF files are stored
PDF_DIR = 'pdfs'

# Define the name of the SQLite database file
DB_FILENAME = 'pdfs.db'

# Define the full path to the database file
DB_PATH = os.path.join(app.root_path, DB_FILENAME)

# Define the database schema
SCHEMA = '''
CREATE TABLE IF NOT EXISTS pdf_files (
    id INTEGER PRIMARY KEY,
    filename TEXT UNIQUE,
    filepath TEXT
);
'''

# Create a connection to the database
conn = sqlite3.connect(DB_PATH)

# Create the pdf_files table if it doesn't already exist
with conn:
    conn.execute(SCHEMA)

# Add all PDF files to the database
for filename in os.listdir(PDF_DIR):
    if filename.endswith('.pdf'):
        filepath = os.path.join(PDF_DIR, filename)
        securefilename = secure_filename(filename)
        with conn:
            conn.execute('INSERT OR IGNORE INTO pdf_files (filename, filepath) VALUES (?, ?);', (securefilename, filepath))

@app.teardown_appcontext
def close_db(error):
    if 'db' in g:
        g.db.close()

def get_db():
    if 'db' not in g:
        g.db = sqlite3.connect('pdfs.db')
    return g.db

# Define the route for serving PDF files
@app.route('/pdfs/<path:filename>')
def serve_pdf(filename):
    return send_from_directory(PDF_DIR, filename)

@app.route('/')
def index():
    return render_template('index.html')

# Define the route for the PDF list page

@app.route('/pdfs')
def pdfs():
    conn = get_db()
    pdf_files = conn.execute('SELECT filename, filepath FROM pdf_files').fetchall()
    return render_template('pdfs.html', pdf_files=pdf_files)

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/contact')
def contact():
    return render_template('contact.html')

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Apr/2023 22:12:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 22:12:08] "GET /pdfs HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 22:12:09] "GET /pdfs/01_Chapter_0_-_Computer_Architecture.pdf HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 22:12:34] "GET /pdfs HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 22:12:35] "GET /pdfs/02_Chapter_1_-_Introduction.pdf HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 22:12:37] "GET /pdfs/09_Chapter_7_-_Modular_Design.pdf HTTP/1.1" 200 -


In [1]:
import os
from werkzeug.utils import secure_filename

PDF_DIR = 'pdfs'

for filename in os.listdir(PDF_DIR):
    if filename.endswith('.pdf'):
        old_path = os.path.join(PDF_DIR, filename)
        new_filename = secure_filename(filename)
        new_path = os.path.join(PDF_DIR, new_filename)
        os.rename(old_path, new_path)
        print(f'Renamed {filename} to {new_filename}')


Renamed 01_Chapter_0_-_Computer_Architecture.pdf to 01_Chapter_0_-_Computer_Architecture.pdf
Renamed 02_Chapter_1_-_Introduction.pdf to 02_Chapter_1_-_Introduction.pdf
Renamed 03_Chapter_2_-_Program_Flowchart.pdf to 03_Chapter_2_-_Program_Flowchart.pdf
Renamed 04_Chapter_3_-_Dry_Running_of_Algorithms.pdf to 04_Chapter_3_-_Dry_Running_of_Algorithms.pdf
Renamed 05_Chapter_4_-_Pseudocoding.pdf to 05_Chapter_4_-_Pseudocoding.pdf
Renamed 06_Chapter_5_-_Descision_Tables.pdf to 06_Chapter_5_-_Descision_Tables.pdf
Renamed 07_Chapter_6_-_Arrays.pdf to 07_Chapter_6_-_Arrays.pdf
Renamed 08_Chapter_7.1_-_Memory_Allocation.pdf to 08_Chapter_7.1_-_Memory_Allocation.pdf
Renamed 09_Chapter_7_-_Modular_Design.pdf to 09_Chapter_7_-_Modular_Design.pdf
Renamed 10_Chapter_8_-_Modularisation.pdf to 10_Chapter_8_-_Modularisation.pdf
Renamed 11_Chapter_9_-_File_Processing.pdf to 11_Chapter_9_-_File_Processing.pdf
Renamed 12_Chapter_10_-_Data_Capturing_and_Correctness.pdf to 12_Chapter_10_-_Data_Capturing_and_

In [2]:
import os
from flask import Flask, render_template, send_from_directory, url_for, g, request
from werkzeug.utils import secure_filename
import sqlite3

app = Flask(__name__)

# Define the directory where PDF files are stored
PDF_DIR = 'pdfs'

# Define the name of the SQLite database file
DB_FILENAME = 'pdfs.db'

# Define the full path to the database file
DB_PATH = os.path.join(app.root_path, DB_FILENAME)

# Define the database schema
SCHEMA = '''
CREATE TABLE IF NOT EXISTS pdf_files (
    id INTEGER PRIMARY KEY,
    filename TEXT UNIQUE,
    filepath TEXT
);
'''

# Create a connection to the database
conn = sqlite3.connect(DB_PATH)

# Create the pdf_files table if it doesn't already exist
with conn:
    conn.execute(SCHEMA)

# Add all PDF files to the database
for filename in os.listdir(PDF_DIR):
    if filename.endswith('.pdf'):
        filepath = os.path.join(PDF_DIR, filename)
        securefilename = secure_filename(filename)
        with conn:
            conn.execute('INSERT OR IGNORE INTO pdf_files (filename, filepath) VALUES (?, ?);', (securefilename, filepath))

@app.teardown_appcontext
def close_db(error):
    if 'db' in g:
        g.db.close()

def get_db():
    if 'db' not in g:
        g.db = sqlite3.connect('pdfs.db')
    return g.db

# Define the route for serving PDF files
@app.route('/pdfs/<path:filename>')
def serve_pdf(filename):
    return send_from_directory(PDF_DIR, filename)

@app.route('/')
def index():
    return render_template('index.html')

# Define the route for the PDF list page

@app.route('/pdfs')
def pdfs():
    # print(request.args)
    # print(request.args.get('query'))
    # print('query' in request.args)
    # print('query')
    print(type('query'))
    if 'query' in request.args:
        query = request.args['query']
        conn = get_db()
        pdf_files = conn.execute('SELECT filename, filepath FROM pdf_files WHERE filename LIKE ?;', ('%' + query + '%', )).fetchall()
        return render_template('pdfs.html', pdf_files=pdf_files)
    else:
        conn = get_db()
        pdf_files = conn.execute('SELECT filename, filepath FROM pdf_files').fetchall()
        return render_template('pdfs.html', pdf_files=pdf_files)

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/contact')
def contact():
    return render_template('contact.html')

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Apr/2023 20:05:18] "GET /pdfs HTTP/1.1" 200 -


ImmutableMultiDict([])
None
False
query
<class 'str'>


127.0.0.1 - - [24/Apr/2023 20:05:23] "GET /pdfs?query=wasd HTTP/1.1" 200 -


ImmutableMultiDict([('query', 'wasd')])
wasd
True
query
<class 'str'>


127.0.0.1 - - [24/Apr/2023 20:05:41] "GET /pdfs?query=h HTTP/1.1" 200 -


ImmutableMultiDict([('query', 'h')])
h
True
query
<class 'str'>


In [7]:
import sqlite3 
conn = sqlite3.connect("pdfs.db")
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS visitor_count (count INT, month INT, year INT)')
conn.commit()
conn.close()

In [9]:
import datetime

now = datetime.datetime.now()
current_month = now.month

print(current_month)

4


In [1]:
import os
from flask import Flask, render_template, send_from_directory, url_for, g, request
from werkzeug.utils import secure_filename
import sqlite3
import datetime

app = Flask(__name__)

# Define the directory where PDF files are stored
PDF_DIR = 'pdfs'

# Define the name of the SQLite database file
DB_FILENAME = 'pdfs.db'

# Define the full path to the database file
DB_PATH = os.path.join(app.root_path, DB_FILENAME)

# Define the database schema
SCHEMA = '''
CREATE TABLE IF NOT EXISTS pdf_files (
    id INTEGER PRIMARY KEY,
    filename TEXT UNIQUE,
    filepath TEXT

);


'''

# Create a connection to the database
conn = sqlite3.connect(DB_PATH)

# Create the pdf_files table if it doesn't already exist
with conn:
    conn.execute(SCHEMA)
    conn.execute("CREATE TABLE IF NOT EXISTS visitor_count (count INT, month INT, year INT);")

# Add all PDF files to the database
for filename in os.listdir(PDF_DIR):
    if filename.endswith('.pdf'):
        filepath = os.path.join(PDF_DIR, filename)
        securefilename = secure_filename(filename)
        with conn:
            conn.execute('INSERT OR IGNORE INTO pdf_files (filename, filepath) VALUES (?, ?);', (securefilename, filepath))

def ordinal(n: int):
    if 11 <= (n % 100) <= 13:
        suffix = 'th'
    else:
        suffix = ['th', 'st', 'nd', 'rd', 'th'][min(n % 10, 4)]
    return str(n) + suffix

@app.teardown_appcontext
def close_db(error):
    if 'db' in g:
        g.db.close()

def get_db():
    if 'db' not in g:
        g.db = sqlite3.connect('pdfs.db')
    return g.db

# Define the route for serving PDF files
@app.route('/pdfs/<path:filename>')
def serve_pdf(filename):
    return send_from_directory(PDF_DIR, filename)

@app.route('/')
def index():
    conn = get_db()
    c = conn.cursor()
    now = datetime.datetime.now()
    current_month = now.month
    current_year = now.year
    c.execute('SELECT count FROM visitor_count WHERE month = ? AND year = ?',(current_month, current_year))
    result = c.fetchone()
    if result:
        count = result[0]
    else:
        count = 0
        c.execute('INSERT INTO visitor_count VALUES(?, ?, ?)', (count, current_month, current_year))
    count +=1
    c.execute('UPDATE visitor_count SET count = ? WHERE month = ? AND year = ?',(count, current_month, current_year))
    conn.commit()
    conn.close()
    count = ordinal(int(count))
    return render_template('index.html', count=count)

# Define the route for the PDF list page

@app.route('/pdfs')
def pdfs():
    print(type('query'))
    if 'query' in request.args:
        query = request.args['query']
        conn = get_db()
        pdf_files = conn.execute('SELECT filename, filepath FROM pdf_files WHERE filename LIKE ? ORDER BY filename ASC;', ('%' + query + '%', )).fetchall()
        # print(pdf_files)
        return render_template('pdfs.html', pdf_files=pdf_files)
    else:
        conn = get_db()
        pdf_files = conn.execute('SELECT filename, filepath FROM pdf_files ORDER BY filename ASC').fetchall()
        return render_template('pdfs.html', pdf_files=pdf_files)

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/contact')
def contact():
    return render_template('contact.html')

if __name__ == '__main__':
    app.run()


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/May/2023 14:24:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 14:24:23] "GET /pdfs HTTP/1.1" 200 -


<class 'str'>


127.0.0.1 - - [10/May/2023 14:24:37] "GET /pdfs?query= HTTP/1.1" 200 -


<class 'str'>
[('01_Chapter_0_-_Computer_Architecture.pdf', 'pdfs/01_Chapter_0_-_Computer_Architecture.pdf'), ('02_Chapter_1_-_Introduction.pdf', 'pdfs/02_Chapter_1_-_Introduction.pdf'), ('03_Chapter_2_-_Program_Flowchart.pdf', 'pdfs/03_Chapter_2_-_Program_Flowchart.pdf'), ('04_Chapter_3_-_Dry_Running_of_Algorithms.pdf', 'pdfs/04_Chapter_3_-_Dry_Running_of_Algorithms.pdf'), ('05_Chapter_4_-_Pseudocoding.pdf', 'pdfs/05_Chapter_4_-_Pseudocoding.pdf'), ('06_Chapter_5_-_Descision_Tables.pdf', 'pdfs/06_Chapter_5_-_Descision_Tables.pdf'), ('07_Chapter_6_-_Arrays.pdf', 'pdfs/07_Chapter_6_-_Arrays.pdf'), ('08_Chapter_7.1_-_Memory_Allocation.pdf', 'pdfs/08_Chapter_7.1_-_Memory_Allocation.pdf'), ('09_Chapter_7_-_Modular_Design.pdf', 'pdfs/09_Chapter_7_-_Modular_Design.pdf'), ('10_Chapter_8_-_Modularisation.pdf', 'pdfs/10_Chapter_8_-_Modularisation.pdf'), ('11_Chapter_9_-_File_Processing.pdf', 'pdfs/11_Chapter_9_-_File_Processing.pdf'), ('12_Chapter_10_-_Data_Capturing_and_Correctness.pdf', 'pdfs

In [3]:

import sqlite3
import datetime
conn = sqlite3.connect("pdfs.db")
c = conn.cursor()
now = datetime.datetime.now()
current_month = now.month
current_year = now.year
c.execute('SELECT count FROM visitor_count WHERE month = ? AND year = ?',(current_month, current_year))
result = c.fetchone()
print(result)
if result:
    count = result[1]
    print("there is result", count)
else:
    count = 0
    c.execute('INSERT INTO visitor_count VALUES(?, ?, ?)', (count, current_month, current_year))
    print("there is no result", count)
count +=1
c.execute('UPDATE visitor_count SET count = ? WHERE month = ? AND year = ?',(count, current_month, current_year))
conn.c

(7,)


IndexError: tuple index out of range

In [5]:
conn = sqlite3.connect("pdf.db")
conn.close()